In [ ]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import pickle
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from IPython.display import display, Image, SVG

from pydrake.examples.quadrotor import QuadrotorGeometry
from pydrake.geometry import MeshcatVisualizerCpp, Rgba, StartMeshcat
from pydrake.geometry.optimization import HPolyhedron, VPolytope
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser
from pydrake.solvers.gurobi import GurobiSolver
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, LeafSystem

from models.building_generation import *
from spp.bezier import BezierSPP
from spp.rounding import *

g_lic = GurobiSolver.AcquireLicense()
m_lic = MosekSolver.AcquireLicense()

In [ ]:
weights = {"time": 1., "norm": 1.}
start = np.array([-1, -1])
goal = np.array([2, 1])

order = 7
continuity = 4
zero_deriv_boundary = 1
vel_limit = 10 * np.ones(3)
start_pose = np.r_[0., 0., 1.]
goal_pose = np.r_[15., 10., 1.]

region_file = "room_debug/regions.reg"
with open(region_file, "rb") as f:
    regions = pickle.load(f)
    
relax_spp = BezierSPP(regions, order, continuity, hdot_min=1e-3)
relax_spp.addTimeCost(weights["time"])
relax_spp.addPathLengthCost(weights["norm"])
relax_spp.addVelocityLimits(-vel_limit, vel_limit)
relax_spp.setPaperSolverOptions()
relax_spp.setSolver(MosekSolver())
relax_spp.addAccelerationRegularization(1e-3, 1e-3)
relax_spp.setRoundingStrategy(averageVertexPositionSpp)

relax_output = relax_spp.SolvePath(start_pose, goal_pose, True, False, zero_deriv_boundary=zero_deriv_boundary)
b_traj_relax, result_relax, best_result_relax, hard_result_relax = relax_output
    
# mip_spp = BezierSPP(regions, order, continuity, hdot_min=1e-3)
# mip_spp.addTimeCost(weights["time"])
# mip_spp.addPathLengthCost(weights["norm"])
# mip_spp.addVelocityLimits(-vel_limit, vel_limit)
# mip_spp.setPaperSolverOptions()
# mip_spp.setSolver(MosekSolver())
# mip_spp.addAccelerationRegularization(1e-3, 1e-3)
# mip_spp.setRoundingStrategy(randomPathSearch)

# mip_output = mip_spp.SolvePath(start_pose, goal_pose, False, False, zero_deriv_boundary=zero_deriv_boundary)
# b_traj_mip, result_mip = mip_output[:2]

In [ ]:
print(result_relax.get_optimal_cost(), best_result_relax.get_optimal_cost()) #, result_mip.get_optimal_cost())